In [ ]:
!pip install tqdm
!pip install dill
!apt install --allow-change-held-packages libcudnn8=8.7.0.84-1+cuda11.8
!pip uninstall -y -q tensorflow-estimator tensorflow-text 
!pip install -U tensorflow-text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libcudnn8 is already the newest version (8.7.0.84-1+cuda11.8).
0 upgraded, 0 newly installed, 0 to remove and 23 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached tensorflow_text-2.12.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (6.0 MB)
  Using cached tensorflow_estimator-2.12.0-py2.py3-none-any.whl (440 kB)


In [ ]:
import dill
import keras

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_text as text

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Embedding, Input, Flatten
from keras.optimizers import Adam

from datetime import datetime

from tqdm.auto import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dialog = pd.read_csv('/content/drive/MyDrive/dialogs.txt', sep='\t')

In [ ]:
dialog.head(5)

,"hi, how are you doing?",i'm fine. how about yourself?
0,i'm fine. how about yourself?,i'm pretty good. thanks for asking.
1,i'm pretty good. thanks for asking.,no problem. so how have you been?
2,no problem. so how have you been?,i've been great. what about you?
3,i've been great. what about you?,i've been good. i'm in school right now.
4,i've been good. i'm in school right now.,what school do you go to?


In [ ]:
dialog = dialog.rename(columns={'hi, how are you doing?': 'Pattern', "i'm fine. how about yourself?": 'Response'})

In [ ]:
dialog.append({'Pattern': 'hi, how are you doing?', 'Response': "i'm fine. how about yourself?"}, ignore_index=True)

<ipython-input-7-b3947e3e654a>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dialog.append({'Pattern': 'hi, how are you doing?', 'Response': "i'm fine. how about yourself?"}, ignore_index=True)


,Pattern,Response
0,i'm fine. how about yourself?,i'm pretty good. thanks for asking.
1,i'm pretty good. thanks for asking.,no problem. so how have you been?
2,no problem. so how have you been?,i've been great. what about you?
3,i've been great. what about you?,i've been good. i'm in school right now.
4,i've been good. i'm in school right now.,what school do you go to?
...,...,...
3720,are you right-handed?,yes. all my life.
3721,yes. all my life.,you're wearing out your right hand. stop using...
3722,you're wearing out your right hand. stop using...,but i do all my writing with my right hand.
3723,but i do all my writing with my right hand.,start typing instead. that way your left hand ...


In [ ]:
X = dialog.Pattern 
y = dialog.Response 

In [ ]:
model_name = "ted_hrlr_translate_pt_en_converter"
tf.keras.utils.get_file(
    f"{model_name}.zip",
    f"https://storage.googleapis.com/download.tensorflow.org/models/{model_name}.zip",
    cache_dir='.', cache_subdir='', extract=True
)

'./ted_hrlr_translate_pt_en_converter.zip'

In [ ]:
tokenizers = tf.saved_model.load('./ted_hrlr_translate_pt_en_converter')

In [ ]:
X = tokenizers.en.tokenize(X)
y = tokenizers.en.tokenize(y)

In [ ]:
X = np.array([tf.reshape(i, [1, -1]) for i in X.to_tensor()])

In [ ]:
y = np.array([tf.reshape(i, [1, -1]) for i in y.to_tensor()])[:, :, :-1]

In [ ]:
X = X[:, 0, :]
y = y[:, 0, :]

In [ ]:
def mask(seq):
  return tf.cast(tf.math.equal(seq, 0), tf.float32)[:, tf.newaxis, tf.newaxis, :]

def mask_final(size):
  return 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)

In [ ]:
class PositionalFF(keras.layers.Layer):
  def __init__(self, d_model, diff):
    super(PositionalFF, self).__init__()

    self.dense = Dense(diff)
    self.relu = keras.layers.ReLU()
    self.dense2 = Dense(d_model)
  
  def call(self, x):
    return self.dense2(self.relu(self.dense(x)))

def positional_encoder(i, d_model):
  position = tf.range(i, dtype=tf.float32)
  position = tf.expand_dims(position, axis=1)
  div_term = tf.pow(10000.0, tf.range(0, d_model, 2, dtype=tf.float32) / d_model)
  div_term = tf.expand_dims(div_term, axis=0)

  sin = tf.sin(position / div_term)
  cos = tf.cos(position / div_term)

  pos_encoding = tf.concat([tf.expand_dims(sin, axis=-1), tf.expand_dims(cos, axis=-1)], axis=-1)
  pos_encoding = tf.reshape(pos_encoding, [-1, i, d_model])

  return pos_encoding

class Add_and_Norm(keras.layers.Layer):
  def __init__(self):
    super(Add_and_Norm, self).__init__()

    self.add = Dropout(.2)
    self.norm = keras.layers.LayerNormalization(epsilon=1e-6)
  
  def call(self, x, y):
    return self.norm(self.add(x) + y)

class Encoder_Layer(keras.layers.Layer):
  def __init__(self, key_size, output_mode, query_size, value_size, num_heads, diff, dropout, bias=False):
    super(Encoder_Layer, self).__init__()

    self.multihead = keras.layers.MultiHeadAttention(num_heads, key_size, value_size, dropout, bias, query_size)
    self.add_norm = Add_and_Norm()
    self.positional_ff = PositionalFF(output_mode, diff)
    self.add_norm_2 = Add_and_Norm()
  
  def call(self, x, padding):
    multiheads = self.multihead(x, x, x, padding)
    addnorm = self.add_norm(multiheads, x)
    pos_ff = self.positional_ff(addnorm)
    x = self.add_norm_2(pos_ff, addnorm)

    return x

class Transformers_Encoder(keras.layers.Layer):
  def __init__(self, vocab_size, key_size, output_mode, query_size, value_size, num_hiddens, num_heads, num_layers, max_, diff, dropout, bias=False):
    super(Transformers_Encoder, self).__init__()

    self.num_hiddens = num_hiddens
    self.max_ = max_
    self.num_layers = num_layers

    self.embeding = Embedding(vocab_size, num_hiddens)
    self.encoder_layer = [Encoder_Layer(key_size, output_mode, query_size, value_size, num_heads, diff, dropout, bias) for _ in range(num_layers)]
    self.dropout = tf.keras.layers.Dropout(dropout)

  def call(self, x, padding): 
    b = tf.shape(x)[1]
    x = self.embeding(x)
    x *= tf.math.sqrt(tf.cast(self.num_hiddens, dtype=tf.float32))
    x = x + positional_encoder(self.max_, self.num_hiddens)[:, :b, :]
   
    x = self.dropout(x)

    for i in range(self.num_layers):
      x = self.encoder_layer[i](x, padding)
    
    return x

In [ ]:
class Decoder_Layer(keras.layers.Layer):
  def __init__(self, key_size, output_mode, query_size, value_size, num_heads, diff, dropout, bias=False):
    super(Decoder_Layer, self).__init__()

    self.multihead = keras.layers.MultiHeadAttention(num_heads, key_size, value_size, dropout, bias, query_size)
    self.add_norm = Add_and_Norm()
    self.positional_ff = PositionalFF(output_mode, diff)
    self.add_norm_2 = Add_and_Norm()
    self.add_norm_3 = Add_and_Norm()
  
  def call(self, x, padding, look_at_head):
    multiheads = self.multihead(x[0], x[0], x[0], look_at_head)
    addnorm = self.add_norm(multiheads, x[0])
    multihead_2 = self.multihead(addnorm, x[1], x[1], padding)
    addnorm_2 = self.add_norm_2(multihead_2, addnorm)
    pos_ff = self.positional_ff(addnorm_2)
    x[0] = self.add_norm_3(pos_ff, addnorm_2)

    return x[0]

class Transformer_Decoder(keras.layers.Layer):
  def __init__(self, vocab_size, key_size, output_mode, query_size, value_size, num_hiddens, num_heads, num_layers, max_, diff, dropout, bias=False):
    super(Transformer_Decoder, self).__init__()

    self.max_ = max_
    self.num_hiddens = num_hiddens
    self.num_layers = num_layers

    self.embeding = Embedding(vocab_size, num_hiddens)

    self.decoder_layer = [Decoder_Layer(key_size, output_mode, query_size, value_size, num_heads, diff, dropout, bias) for _ in range(num_layers)]

    self.dropout = tf.keras.layers.Dropout(dropout)
  
  def call(self, x, padding, look_at_head):
    b = tf.shape(x[0])[1]

    x_ = self.embeding(x[0])
    x_ *= tf.math.sqrt(tf.cast(self.num_hiddens, dtype=tf.float32))
    x_ = x_ + positional_encoder(self.max_, self.num_hiddens)[:, :b, :]

    x_ =  self.dropout(x_)

    for i in range(self.num_layers):
      x_ = self.decoder_layer[i]([x_, x[1]], padding, look_at_head)
    
    return x_

In [ ]:
# test
dataset = np.array([[0,1,2, 5], [2, 3, 5, 5]]).astype('float32')
te = Transformers_Encoder(len(dataset), 1, 4, 4, 1 ,4, 5, 10, 4, 512, .3, False)

all = te(dataset, mask(dataset), training=False)

In [ ]:
all

<tf.Tensor: shape=(2, 4, 4), dtype=float32, numpy=
array([[[-1.3788465 ,  1.0927151 , -0.5141319 ,  0.8002633 ],
        [-1.1749277 ,  0.94803935, -0.8071599 ,  1.0340483 ],
        [-1.0834072 ,  0.5147339 , -0.8110126 ,  1.3796859 ],
        [-1.3579271 ,  0.40585077, -0.40091437,  1.3529906 ]],

       [[-1.0252202 ,  1.1535561 , -0.96138424,  0.83304816],
        [-0.67885864,  1.0248681 , -1.2750329 ,  0.9290234 ],
        [-0.5922313 ,  0.48541874, -1.2519524 ,  1.358765  ],
        [-1.0061094 ,  0.38123596, -0.83803403,  1.4629076 ]]],
      dtype=float32)>

In [ ]:
decoder_blk = Transformer_Decoder(len(dataset), 1, 4, 4, 1 ,4, 5, 10, 4, 512, .3, True)

all_2 = decoder_blk([dataset, all],None, None, training=False)[0].numpy()

In [ ]:
all_2

array([[-0.10101867,  0.57299685, -1.5682962 ,  1.0963179 ],
       [ 0.99319416, -0.20527668, -1.547435  ,  0.7595176 ],
       [ 1.1095284 , -0.9958678 , -0.9977687 ,  0.8841081 ],
       [ 0.75946045, -1.1084163 , -0.8584169 ,  1.2073729 ]],
      dtype=float32)

In [ ]:
num_hiddens, num_blks, dropout = 128, 2, .1
num_heads, max_, num_layers = 8, 10000, 4
key_size, query_size, value_size = 128, 128, 128
output_mode, diff = 128, 512
shape = 8500
use_bias = True

class Transformer(Model):
  def __init__(self):
    super(Transformer, self).__init__()
    self.encoder = Transformers_Encoder(shape, key_size, output_mode, query_size, value_size, num_hiddens, num_heads, num_layers, max_, diff, dropout, use_bias)
    self.decoder = Transformer_Decoder(shape, key_size, output_mode, query_size, value_size, num_hiddens, num_heads, num_layers, max_, diff, dropout, use_bias)
    self.dense = Dense(shape, activation='softmax')

  def create_masks(self, inp, tar):
    enc_padding_mask = mask(inp)

    dec_padding_mask = mask(inp)

    look_ahead_mask = mask_final(tf.shape(tar)[1])
    dec_target_padding_mask = mask(tar)
    look_ahead_mask = tf.maximum(dec_target_padding_mask, look_ahead_mask)

    return enc_padding_mask, look_ahead_mask, dec_padding_mask

  def call(self, x):
    x, y = x

    padding_for_encoder, look_at_head, padding_for_decoder = self.create_masks(x, y)
    
    encoder_ = self.encoder(x, padding_for_encoder)
    return self.dense(self.decoder([y, encoder_], padding_for_decoder, look_at_head))

transformer = Transformer()

In [ ]:
reshape = tf.reshape(y[1], [1, -1])
reshape_2 = tf.reshape(X[1], [1, -1])

result = transformer([reshape, reshape_2])

In [ ]:
result.shape

TensorShape([1, 27, 8500])

In [ ]:
loss_object = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
accuracy = tf.metrics.SparseCategoricalCrossentropy('accuracy')

def loss_transformers(y_real, y_pred):
  mask = tf.math.logical_not(tf.math.equal(y_real, 0))
  
  loss = loss_object(y_real, y_pred)

  mask = tf.cast(mask, dtype=loss.dtype)
  loss *= mask

  return tf.reduce_sum(loss)/tf.reduce_sum(mask)

optimaizer = tf.keras.optimizers.Adam(3e-5, beta_1=0.9, beta_2=0.98,
                                     epsilon=1e-9)

train_step_signature = [
    tf.TensorSpec(shape=(None, None), dtype=tf.int64),
    tf.TensorSpec(shape=(None, None), dtype=tf.int64),
]


@tf.function(input_signature=train_step_signature)
def train(x, y):
  input_y = y[:, :-1]
  real_y = y[:, 1:]

  with tf.GradientTape() as transformers_tape:
    result = transformer([x, input_y], training=True)

    loss = loss_transformers(real_y, result)

  tape = transformers_tape.gradient(loss, transformer.trainable_variables)

  optimaizer.apply_gradients(zip(tape, transformer.trainable_variables))
  
  accuracy.update_state(real_y, result)

  return loss
  
loss_transformers(reshape, result)

/usr/local/lib/python3.9/dist-packages/keras/backend.py:5612: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


<tf.Tensor: shape=(), dtype=float32, numpy=9.163382>

In [ ]:
def absolute_training(epchos):
  for _ in range(epchos):
    loss_arr = []

    for i in tqdm(range(len(dialog))):
      x_ = tf.reshape(X[i], [1, -1])
      y_ = tf.reshape(y[i], [1, -1])

      loss = train(x_, y_)
      loss_arr.append(loss.numpy())

    lossing = sum(loss_arr) / len(loss_arr)

    print(f"epochs - {_+1}\nloss - {lossing}, acc - {accuracy.result().numpy()}")
    accuracy.reset_states()

In [ ]:
absolute_training(3)

  0%|          | 0/3724 [00:00<?, ?it/s]

epochs - 1
loss - 2.993393732056774, acc - 2.993393898010254


  0%|          | 0/3724 [00:00<?, ?it/s]

epochs - 2
loss - 1.6313840414684517, acc - 1.6313807964324951


  0%|          | 0/3724 [00:00<?, ?it/s]

epochs - 3
loss - 1.3362122656293334, acc - 1.3362106084823608


In [38]:
while True:
  inp = input('You - ')

  if inp.lower() == 'break':
    break
  
  me = tokenizers.en.tokenize([inp]).numpy()

  result = transformer([me, me[:, :-1]])

  result = tf.constant(np.array([[i.argmax() for i in result[0].numpy()]]))

  print(f'Omega - {tokenizers.en.detokenize(result)[0].numpy().decode("utf-8")}')

You - hello
Omega - he in
You - how are you doing now
Omega - no are you have is and
You - okay. who are you?
Omega - yes . yes are ? ?
You - break
